<a href="https://colab.research.google.com/github/alyagabsi/inventory-prediction/blob/main/Optimization_Section/Order_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>